In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
# Set search terms and date range
search_terms = 'Ukraine'
start_date = '2021-01-01'
end_date = '2021-12-31'


In [ ]:
# Set up search query
query = f'{search_terms} since:{start_date} until:{end_date}'

In [ ]:
# Use snscrape to get up to 10000 tweets matching the search query
tweets = []
counter = 0
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    # Estimate the view count based on retweets and likes
    view_count = tweet.likeCount + tweet.retweetCount * 5
    if view_count >= 100:
        tweets.append({
            'id': tweet.id,
            'date': tweet.date,
            'content': tweet.content,
            'view_count': view_count,
            'likes': tweet.likeCount,
            'retweets': tweet.retweetCount,
            'replies': tweet.replyCount,
            'impressions': tweet.structured_metrics['impressionCount'],
            'clicks': tweet.structured_metrics['urlLinkClicks'],
            'conversions': tweet.structured_metrics['userProfileClicks'],
            'sentiment': 0  # Add code here to calculate sentiment
        })
        counter += 1
        if counter == 10000:
            break

In [ ]:
# Convert tweets to a pandas DataFrame
df = pd.DataFrame(tweets)

In [ ]:
# Define the weights for each metric
engagement_weight = 0.4
impressions_weight = 0.2
clicks_weight = 0.2
conversions_weight = 0.1
sentiment_weight = 0.1

In [ ]:
# Scale the metrics to a range of 0-1
df['engagement_score'] = df['likes'] + df['retweets'] + df['replies']
df['impressions_score'] = df['impressions']
df['clicks_score'] = df['clicks']
df['conversions_score'] = df['conversions']
df['sentiment_score'] = (df['sentiment'] + 1) / 2
df['engagement_score'] = df['engagement_score'] / df['engagement_score'].max()
df['impressions_score'] = df['impressions_score'] / df['impressions_score'].max()
df['clicks_score'] = df['clicks_score'] / df['clicks_score'].max()
df['conversions_score'] = df['conversions_score'] / df['conversions_score'].max()

In [ ]:
# Calculate the overall index for each tweet
df['index'] = (
    df['engagement_score'] * engagement_weight +
    df['impressions_score'] * impressions_weight +
    df['clicks_score'] * clicks_weight +
    df['conversions_score'] * conversions_weight +
    df['sentiment_score'] * sentiment_weight
)

In [ ]:
# Sort the tweets by the index, in descending order
df = df.sort_values(by='index', ascending=False)

In [ ]:
# Output the top 10 tweets by the index
print(df[['content', 'index']].head(10))